<a href="https://colab.research.google.com/github/Aditya-B-007/Chatbot/blob/main/BERT_email_spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install tensorflow-text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.7 MB/s eta 0:00:00


In [10]:
import pandas as pd
df=pd.read_csv('/content/spam_ham_dataset.csv')
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [11]:
df.groupby('label').describe()
#clearly we can see the count of the authentic emails are 3672 and spam emails are 1499. How we will classify them is what I shall do

Unnamed: 0                                                         \
           count    mean          std     min      25%     50%      75%   
label                                                                     
ham       3672.0  1835.5  1060.159422     0.0   917.75  1835.5  2753.25   
spam      1499.0  4421.0   432.868340  3672.0  4046.50  4421.0  4795.50   

              label_num                                     
          max     count mean  std  min  25%  50%  75%  max  
label                                                       
ham    3671.0    3672.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
spam   5170.0    1499.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

In [12]:
df['spam']=df['label'].apply(lambda x: 1 if x=='spam' else 0)
df['ham']=df['label'].apply(lambda x: 1 if x=='ham' else 0)
df.head()

,Unnamed: 0,label,text,label_num,spam,ham
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0,0,1
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,0,1
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,0,1
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1,1,0
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0,0,1


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['text'],df['spam'],test_size=0.2)
#We have divided the datasets into training and testing.

In [14]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [15]:
#what we did here is that we checked if the bert classification runs properly with the bert, and it looks like it does so we can go ahead now
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding(["Congratulations! You've won a $1,000 gift card. Click here to claim your prize.",
                       "Your account has been compromised. Please click the link below to verify your identity.",
                       "You have been selected for a special offer. Buy one get one free. Click here for more details.",
                       "Aditya, you would be having a meeting every friday"])

<tf.Tensor: shape=(4, 768), dtype=float32, numpy=
array([[-0.7726122 , -0.4348053 , -0.9085034 , ..., -0.6997763 ,
        -0.6959363 ,  0.7531596 ],
       [-0.831194  , -0.53914744, -0.8866283 , ..., -0.65568644,
        -0.7598695 ,  0.7706839 ],
       [-0.89384484, -0.57053   , -0.96976525, ..., -0.901958  ,
        -0.76907504,  0.84563357],
       [-0.5656236 , -0.2931865 , -0.67688006, ..., -0.6327342 ,
        -0.52115846,  0.5935337 ]], dtype=float32)>

In [17]:
#I have defined the bert layers here for thee text classification
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

 #developing a neural network
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
#defining the model which would be our NN
text_classifier = tf.keras.Model(inputs=[text_input], outputs = [l])

In [18]:
text_classifier.summary() #summarise the model

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_word_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                              

In [21]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'), #Focussing on binary accuracy
      tf.keras.metrics.Precision(name='precision'), #Focusing on precision
      tf.keras.metrics.Recall(name='recall')#what is recall ? recall is asking if the model got all the spam cases
]

text_classifier.compile(optimizer='adam', # Adam propagation is one of the best thus chose it
              loss='binary_crossentropy', # The reason I chose binary crossentropy was that we have only two categories, spam and ham which we have to detect and classify
              metrics=METRICS)

In [24]:
text_classifier.fit(x_train, y_train, epochs=1)

130/130 [==============================] - 1829s 14s/step - loss: 0.4870 - accuracy: 0.7790 - precision: 0.7526 - recall: 0.3645


In [25]:
text_classifier.evaluate(x_test, y_test)

33/33 [==============================] - 455s 14s/step - loss: 0.4422 - accuracy: 0.8174 - precision: 0.9032 - recall: 0.3875


[0.44216224551200867,
 0.8173912763595581,
 0.9032257795333862,
 0.387543261051178]

In [26]:
y_predicted = text_classifier.predict(x_test)
y_predicted = y_predicted.flatten()

33/33 [==============================] - 456s 14s/step


In [27]:
y_predicted = text_classifier.predict(x_test)
y_predicted = y_predicted.flatten()

33/33 [==============================] - 485s 15s/step


In [29]:
submission=pd.DataFrame({'spam':range(1,len(y_predicted)+1),'label':y_predicted})
submission.to_csv('spam_detected.csv',index=False)